# MTLR + 5-Model Ensemble (Independent Seeds & Hyperparameters)

- Model: MTLR (torchmtlr)
- Ensemble: 5 models, each with its own Optuna optimization (different seeds)
- Metric: IPCW C-index (sksurv)
- Validation: nested CV (outer for evaluation, inner for hyperparameter tuning)
- Ensembling: rank-based aggregation of risk scores
- Statistical evaluation: mean, std, 95% CI, paired t-test (ensemble vs single model)

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torchmtlr import MTLR, mtlr_neg_log_likelihood
from torchmtlr.utils import encode_survival, make_time_bins

from sksurv.util import Surv
from sksurv.metrics import concordance_index_ipcw

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import optuna
import matplotlib.pyplot as plt
from scipy import stats

# Global seeds
torch.manual_seed(42)
np.random.seed(42)

# Main configuration
N_MODELS_ENSEMBLE = 5        # number of models in the ensemble
N_SPLITS_OUTER = 5           # number of outer folds
N_SPLITS_INNER = 3           # number of inner folds (Optuna CV)
N_TRIALS_INNER = 30          # number of Optuna trials per model per outer fold
TAU_CINDEX = 7.0             # horizon for IPCW C-index

# 1. Data Loading & Preprocessing

In [ ]:
df_train = pd.read_csv("../../data/train_enhanced.csv")
df_eval = pd.read_csv("../../data/eval_enhanced.csv")

print(f"Train data: {df_train.shape}")
print(f"Eval data : {df_eval.shape}")
print("\nTrain columns:")
print(df_train.columns.tolist())

In [ ]:
target_cols = ["OS_STATUS", "OS_YEARS"]

# Train features
X = df_train.drop(columns=target_cols + ["ID"])
X = pd.get_dummies(X, drop_first=True).astype(float)

# Eval features (aligned with train)
X_eval = df_eval.drop(columns=["ID"])
X_eval = pd.get_dummies(X_eval, drop_first=True)
X_eval = X_eval.reindex(columns=X.columns, fill_value=0).astype(float)

# Standardization
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns,
    index=X.index,
)
X_eval_scaled = pd.DataFrame(
    scaler.transform(X_eval),
    columns=X_eval.columns,
    index=X_eval.index,
)

X = X_scaled.fillna(0)
X_eval = X_eval_scaled.fillna(0)

# Targets (torch tensors)
y_time = torch.tensor(df_train["OS_YEARS"].values, dtype=torch.float32)
y_event = torch.tensor(df_train["OS_STATUS"].values, dtype=torch.float32)

print(f"X shape      : {X.shape}")
print(f"X_eval shape : {X_eval.shape}")
print(f"NaN in X      : {X.isna().sum().sum()}")
print(f"NaN in X_eval : {X_eval.isna().sum().sum()}")

# 2. MTLR Utilities & Risk Scoring

In [ ]:
def set_global_seeds(seed: int):
    """Set NumPy and PyTorch seeds for reproducibility."""
    np.random.seed(seed)
    torch.manual_seed(seed)


def build_mtlr_model(input_dim: int, time_bins, params: dict) -> nn.Module:
    """Build an MTLR neural network from a hyperparameter dictionary."""
    if params["activation"] == "relu":
        activation = nn.ReLU()
    elif params["activation"] == "leaky_relu":
        activation = nn.LeakyReLU()
    else:
        activation = nn.ELU()

    model = nn.Sequential(
        nn.Linear(input_dim, params["n_hidden1"]),
        nn.BatchNorm1d(params["n_hidden1"]),
        activation,
        nn.Dropout(params["dropout1"]),
        nn.Linear(params["n_hidden1"], params["n_hidden2"]),
        nn.BatchNorm1d(params["n_hidden2"]),
        activation,
        nn.Dropout(params["dropout2"]),
        MTLR(params["n_hidden2"], len(time_bins)),
    )
    return model


def get_optimizer(model: nn.Module, params: dict):
    """Return an optimizer configured with the given hyperparameters."""
    if params["optimizer"] == "adamw":
        return torch.optim.AdamW(
            model.parameters(),
            lr=params["lr"],
            weight_decay=params["weight_decay"],
        )
    else:
        return torch.optim.Adam(
            model.parameters(),
            lr=params["lr"],
            weight_decay=params["weight_decay"],
        )


def train_mtlr_model(
    model: nn.Module,
    optimizer,
    X_tensor: torch.Tensor,
    y_time_fold: torch.Tensor,
    y_event_fold: torch.Tensor,
    time_bins,
    C1: float,
    n_epochs: int,
    clip_grad: float = 1.0,
    verbose: bool = False,
):
    """
    Train an MTLR model on a given dataset.
    Returns the trained model and the list of losses.
    If NaN occurs in training loss, returns (None, None).
    """
    target = encode_survival(y_time_fold, y_event_fold, time_bins)
    losses = []

    model.train()
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        logits = model(X_tensor)
        loss = mtlr_neg_log_likelihood(
            logits, target, model[-1], C1=C1, average=True
        )

        if torch.isnan(loss):
            # signal failure
            return None, None

        loss.backward()
        if clip_grad is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad)
        optimizer.step()
        losses.append(loss.item())

        if verbose and ((epoch + 1) % max(1, n_epochs // 5) == 0):
            print(f"  Epoch {epoch+1}/{n_epochs} - Loss: {loss.item():.4f}")

    return model, losses


def mtlr_risk_scores(model: nn.Module, X_tensor: torch.Tensor) -> np.ndarray:
    """
    Compute MTLR risk scores using log-sum-exp over logits.
    Higher score = higher risk.
    """
    model.eval()
    with torch.no_grad():
        logits = model(X_tensor)
        risk = torch.logsumexp(logits, dim=1).cpu().numpy()
    return risk


def rank_based_ensemble(risk_matrix: np.ndarray) -> np.ndarray:
    """
    Rank-based ensembling:
      - risk_matrix: shape (n_models, n_samples)
      - For each model, convert risk scores to ranks (1 = lowest risk, N = highest risk),
        then average ranks across models.
    """
    n_models, n_samples = risk_matrix.shape
    ranks_all = np.zeros_like(risk_matrix, dtype=float)

    for m in range(n_models):
        scores = risk_matrix[m]
        order = np.argsort(scores)  # ascending
        ranks = np.empty_like(order, dtype=float)
        ranks[order] = np.arange(1, n_samples + 1)
        ranks_all[m] = ranks

    mean_ranks = ranks_all.mean(axis=0)
    return mean_ranks


def compute_ipcw_cindex(y_train_struct, y_test_struct, risk_scores, tau: float):
    """Compute IPCW C-index on test data."""
    return concordance_index_ipcw(
        y_train_struct,
        y_test_struct,
        risk_scores,
        tau=tau,
    )[0]

# 3. Nested Cross-Validation: Outer (Evaluation) / Inner (Optuna) + 5-Model Ensemble

In [ ]:
outer_kf = KFold(n_splits=N_SPLITS_OUTER, shuffle=True, random_state=123)

cv_single_scores = []    # single model = first model in the ensemble
cv_ensemble_scores = []  # ensemble of 5 models (rank-based)

print("=== Starting outer cross-validation ===")

for fold, (train_idx, test_idx) in enumerate(outer_kf.split(X), 1):
    print(f"\n######## Outer Fold {fold}/{N_SPLITS_OUTER} ########")

    # --- Outer fold data ---
    X_train_outer = X.iloc[train_idx].reset_index(drop=True)
    X_test_outer = X.iloc[test_idx].reset_index(drop=True)

    y_time_train_outer = y_time[train_idx]
    y_time_test_outer = y_time[test_idx]
    y_event_train_outer = y_event[train_idx]
    y_event_test_outer = y_event[test_idx]

    X_train_outer_tensor = torch.tensor(
        X_train_outer.values, dtype=torch.float32
    )
    X_test_outer_tensor = torch.tensor(
        X_test_outer.values, dtype=torch.float32
    )

    time_bins_outer = make_time_bins(y_time_train_outer, event=y_event_train_outer)

    # Surv structures for outer C-index
    y_train_struct_outer = Surv.from_arrays(
        event=y_event_train_outer.numpy().astype(bool),
        time=y_time_train_outer.numpy(),
    )
    y_test_struct_outer = Surv.from_arrays(
        event=y_event_test_outer.numpy().astype(bool),
        time=y_time_test_outer.numpy(),
    )

    # --- Ensemble: train 5 independent models on this outer fold ---
    risk_models_test = []
    single_cindex_this_fold = None  # C-index of model 0 (baseline single)

    for m in range(N_MODELS_ENSEMBLE):
        print(f"\n  >> Ensemble model {m+1}/{N_MODELS_ENSEMBLE} (outer fold {fold})")

        # Base seed for this model & fold
        base_seed = 1000 + 100 * fold + m
        set_global_seeds(base_seed)

        # === INNER CV: Hyperparameter optimization on X_train_outer ===
        inner_kf = KFold(
            n_splits=N_SPLITS_INNER, shuffle=True, random_state=base_seed
        )

        def inner_objective(trial):
            # Hyperparameter search space
            params = {
                "n_hidden1": trial.suggest_int("n_hidden1", 32, 256, step=32),
                "n_hidden2": trial.suggest_int("n_hidden2", 16, 128, step=16),
                "dropout1": trial.suggest_float("dropout1", 0.0, 0.5),
                "dropout2": trial.suggest_float("dropout2", 0.0, 0.5),
                "lr": trial.suggest_float("lr", 1e-4, 1e-2, log=True),
                "n_epochs": trial.suggest_int("n_epochs", 50, 200, step=25),
                "C1": trial.suggest_float("C1", 0.1, 5.0, log=True),
                "activation": trial.suggest_categorical(
                    "activation", ["relu", "leaky_relu", "elu"]
                ),
                "optimizer": trial.suggest_categorical(
                    "optimizer", ["adam", "adamw"]
                ),
                "weight_decay": trial.suggest_float(
                    "weight_decay", 1e-6, 1e-3, log=True
                ),
            }

            inner_scores = []

            for inner_fold, (inner_train_idx, inner_val_idx) in enumerate(
                inner_kf.split(X_train_outer), 1
            ):
                set_global_seeds(base_seed + trial.number * 10 + inner_fold)

                X_train_inner = X_train_outer.iloc[inner_train_idx]
                X_val_inner = X_train_outer.iloc[inner_val_idx]

                y_time_train_inner = y_time_train_outer[inner_train_idx]
                y_time_val_inner = y_time_train_outer[inner_val_idx]
                y_event_train_inner = y_event_train_outer[inner_train_idx]
                y_event_val_inner = y_event_train_outer[inner_val_idx]

                X_train_inner_tensor = torch.tensor(
                    X_train_inner.values, dtype=torch.float32
                )
                X_val_inner_tensor = torch.tensor(
                    X_val_inner.values, dtype=torch.float32
                )

                time_bins_inner = make_time_bins(
                    y_time_train_inner, event=y_event_train_inner
                )

                model_inner = build_mtlr_model(
                    input_dim=X_train_inner.shape[1],
                    time_bins=time_bins_inner,
                    params=params,
                )
                optimizer_inner = get_optimizer(model_inner, params)

                model_inner, _ = train_mtlr_model(
                    model=model_inner,
                    optimizer=optimizer_inner,
                    X_tensor=X_train_inner_tensor,
                    y_time_fold=y_time_train_inner,
                    y_event_fold=y_event_train_inner,
                    time_bins=time_bins_inner,
                    C1=params["C1"],
                    n_epochs=params["n_epochs"],
                    clip_grad=1.0,
                    verbose=False,
                )

                if model_inner is None:
                    inner_scores.append(0.5)
                    continue

                y_train_struct_inner = Surv.from_arrays(
                    event=y_event_train_inner.numpy().astype(bool),
                    time=y_time_train_inner.numpy(),
                )
                y_val_struct_inner = Surv.from_arrays(
                    event=y_event_val_inner.numpy().astype(bool),
                    time=y_time_val_inner.numpy(),
                )

                risk_val_inner = mtlr_risk_scores(
                    model_inner, X_val_inner_tensor
                )

                if np.isnan(risk_val_inner).any():
                    inner_scores.append(0.5)
                    continue

                try:
                    cindex_inner = compute_ipcw_cindex(
                        y_train_struct_inner,
                        y_val_struct_inner,
                        risk_val_inner,
                        tau=TAU_CINDEX,
                    )
                except Exception:
                    cindex_inner = 0.5

                inner_scores.append(cindex_inner)

            return float(np.mean(inner_scores))

        # Optuna study for this model & this outer fold
        study = optuna.create_study(
            direction="maximize",
            study_name=f"mtlr_outer_fold{fold}_model{m}",
        )
        study.optimize(
            inner_objective,
            n_trials=N_TRIALS_INNER,
            show_progress_bar=False,
        )

        best_params = study.best_params
        print(f"    - Best inner CV C-index : {study.best_value:.4f}")
        print(f"    - Best params           : {best_params}")

        # === Final training of this model on full outer-train set ===
        model_m = build_mtlr_model(
            input_dim=X_train_outer.shape[1],
            time_bins=time_bins_outer,
            params=best_params,
        )
        optimizer_m = get_optimizer(model_m, best_params)

        model_m, _ = train_mtlr_model(
            model=model_m,
            optimizer=optimizer_m,
            X_tensor=X_train_outer_tensor,
            y_time_fold=y_time_train_outer,
            y_event_fold=y_event_train_outer,
            time_bins=time_bins_outer,
            C1=best_params["C1"],
            n_epochs=best_params["n_epochs"],
            clip_grad=1.0,
            verbose=False,
        )

        if model_m is None:
            print("    ⚠ Training failed on outer train set. Model ignored.")
            continue

        # Risk scores on outer test
        risk_test_m = mtlr_risk_scores(model_m, X_test_outer_tensor)
        risk_models_test.append(risk_test_m)

        # Define "single model" as the first model in the ensemble
        if m == 0:
            if np.isnan(risk_test_m).any():
                cindex_single = 0.5
            else:
                cindex_single = compute_ipcw_cindex(
                    y_train_struct_outer,
                    y_test_struct_outer,
                    risk_test_m,
                    tau=TAU_CINDEX,
                )
            single_cindex_this_fold = cindex_single
            print(f"    -> Single model C-index on outer test : {cindex_single:.4f}")

    # End of the loop over ensemble models for this outer fold

    # If no valid models:
    if len(risk_models_test) == 0:
        print("  ⚠ No valid models for this fold. Ensemble C-index = 0.5.")
        cindex_ensemble = 0.5
    else:
        risk_matrix_test = np.vstack(risk_models_test)
        risk_ensemble = rank_based_ensemble(risk_matrix_test)
        if np.isnan(risk_ensemble).any():
            cindex_ensemble = 0.5
        else:
            cindex_ensemble = compute_ipcw_cindex(
                y_train_struct_outer,
                y_test_struct_outer,
                risk_ensemble,
                tau=TAU_CINDEX,
            )

    # If single model did not compute (e.g. model 0 failed)
    if single_cindex_this_fold is None:
        single_cindex_this_fold = 0.5

    cv_single_scores.append(single_cindex_this_fold)
    cv_ensemble_scores.append(cindex_ensemble)

    print(f"\n  Fold {fold} summary:")
    print(f"    Single model C-index : {single_cindex_this_fold:.4f}")
    print(f"    Ensemble C-index     : {cindex_ensemble:.4f}")

# 4. Statistical Analysis: Means, 95% CI, Paired t-test

In [ ]:
def summarize_scores(scores, name: str):
    scores = np.array(scores)
    mean = scores.mean()
    std = scores.std(ddof=1)
    n = len(scores)

    t_crit = stats.t.ppf(0.975, df=n - 1)
    ci_low = mean - t_crit * std / np.sqrt(n)
    ci_high = mean + t_crit * std / np.sqrt(n)

    print(f"\n{name}")
    print(f"  Scores by fold : {np.round(scores, 4)}")
    print(f"  Mean           : {mean:.4f}")
    print(f"  Std dev        : {std:.4f}")
    print(f"  95% CI         : [{ci_low:.4f} ; {ci_high:.4f}]")
    return scores, mean, std, (ci_low, ci_high)


single_scores, single_mean, single_std, single_ci = summarize_scores(
    cv_single_scores, "Single model (first model of ensemble)"
)
ensemble_scores, ensemble_mean, ensemble_std, ensemble_ci = summarize_scores(
    cv_ensemble_scores, "Ensemble (5 models, rank-based)"
)

# Paired t-test: ensemble vs single
diff = ensemble_scores - single_scores
t_stat, p_val = stats.ttest_rel(ensemble_scores, single_scores)

print("\nPaired t-test: Ensemble vs Single model")
print(f"  Fold-wise differences (ensemble - single) : {np.round(diff, 4)}")
print(f"  Mean difference : {diff.mean():.4f}")
print(f"  t-statistic     : {t_stat:.4f}")
print(f"  p-value         : {p_val:.4f}")
if p_val < 0.05:
    print("  => Statistically significant improvement at 5% level")
else:
    print("  => NOT statistically significant at 5% level with these folds")

# Visualization
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
folds = np.arange(1, N_SPLITS_OUTER + 1)
plt.plot(folds, single_scores, "o-", label="Single model", linewidth=2, markersize=8)
plt.plot(folds, ensemble_scores, "s-", label="Ensemble (rank)", linewidth=2, markersize=8)
plt.xlabel("Outer fold")
plt.ylabel("IPCW C-index")
plt.title("C-index per outer fold")
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 2, 2)
plt.bar(["Single", "Ensemble"], [single_mean, ensemble_mean],
        yerr=[single_std, ensemble_std], capsize=5, alpha=0.7)
plt.ylabel("IPCW C-index (mean)")
plt.title("Average performance (±1 SD)")

plt.tight_layout()
plt.show()

# 5. Final Training on Full Train Set + 5-Model Ensemble for Evaluation Predictions

In [ ]:
def optimize_and_train_full_model(X_data, y_time_data, y_event_data, base_seed: int):
    """
    On the full dataset:
      - Inner CV to optimize hyperparameters (with Optuna)
      - Final training on the full data with best hyperparameters
    Returns the trained model, time bins, best params, and loss history.
    """
    set_global_seeds(base_seed)

    X_data = X_data.reset_index(drop=True)
    X_tensor = torch.tensor(X_data.values, dtype=torch.float32)
    time_bins_data = make_time_bins(y_time_data, event=y_event_data)

    inner_kf = KFold(
        n_splits=N_SPLITS_INNER, shuffle=True, random_state=base_seed
    )

    def inner_objective_full(trial):
        params = {
            "n_hidden1": trial.suggest_int("n_hidden1", 32, 256, step=32),
            "n_hidden2": trial.suggest_int("n_hidden2", 16, 128, step=16),
            "dropout1": trial.suggest_float("dropout1", 0.0, 0.5),
            "dropout2": trial.suggest_float("dropout2", 0.0, 0.5),
            "lr": trial.suggest_float("lr", 1e-4, 1e-2, log=True),
            "n_epochs": trial.suggest_int("n_epochs", 50, 200, step=25),
            "C1": trial.suggest_float("C1", 0.1, 5.0, log=True),
            "activation": trial.suggest_categorical(
                "activation", ["relu", "leaky_relu", "elu"]
            ),
            "optimizer": trial.suggest_categorical(
                "optimizer", ["adam", "adamw"]
            ),
            "weight_decay": trial.suggest_float(
                "weight_decay", 1e-6, 1e-3, log=True
            ),
        }

        inner_scores = []

        for inner_fold, (inner_train_idx, inner_val_idx) in enumerate(
            inner_kf.split(X_data), 1
        ):
            set_global_seeds(base_seed + trial.number * 10 + inner_fold)

            X_train_inner = X_data.iloc[inner_train_idx]
            X_val_inner = X_data.iloc[inner_val_idx]

            y_time_train_inner = y_time_data[inner_train_idx]
            y_time_val_inner = y_time_data[inner_val_idx]
            y_event_train_inner = y_event_data[inner_train_idx]
            y_event_val_inner = y_event_data[inner_val_idx]

            X_train_inner_tensor = torch.tensor(
                X_train_inner.values, dtype=torch.float32
            )
            X_val_inner_tensor = torch.tensor(
                X_val_inner.values, dtype=torch.float32
            )

            time_bins_inner = make_time_bins(
                y_time_train_inner, event=y_event_train_inner
            )

            model_inner = build_mtlr_model(
                input_dim=X_train_inner.shape[1],
                time_bins=time_bins_inner,
                params=params,
            )
            optimizer_inner = get_optimizer(model_inner, params)

            model_inner, _ = train_mtlr_model(
                model=model_inner,
                optimizer=optimizer_inner,
                X_tensor=X_train_inner_tensor,
                y_time_fold=y_time_train_inner,
                y_event_fold=y_event_train_inner,
                time_bins=time_bins_inner,
                C1=params["C1"],
                n_epochs=params["n_epochs"],
                clip_grad=1.0,
                verbose=False,
            )

            if model_inner is None:
                inner_scores.append(0.5)
                continue

            y_train_struct_inner = Surv.from_arrays(
                event=y_event_train_inner.numpy().astype(bool),
                time=y_time_train_inner.numpy(),
            )
            y_val_struct_inner = Surv.from_arrays(
                event=y_event_val_inner.numpy().astype(bool),
                time=y_time_val_inner.numpy(),
            )

            risk_val_inner = mtlr_risk_scores(model_inner, X_val_inner_tensor)

            if np.isnan(risk_val_inner).any():
                inner_scores.append(0.5)
                continue

            try:
                cindex_inner = compute_ipcw_cindex(
                    y_train_struct_inner,
                    y_val_struct_inner,
                    risk_val_inner,
                    tau=TAU_CINDEX,
                )
            except Exception:
                cindex_inner = 0.5

            inner_scores.append(cindex_inner)

        return float(np.mean(inner_scores))

    # Optuna on full dataset (inner CV)
    study_full = optuna.create_study(
        direction="maximize",
        study_name=f"mtlr_full_seed{base_seed}",
    )
    study_full.optimize(
        inner_objective_full,
        n_trials=N_TRIALS_INNER,
        show_progress_bar=False,
    )

    best_params_full = study_full.best_params
    print(f"Seed {base_seed} - best inner CV C-index: {study_full.best_value:.4f}")
    print(f"Best params: {best_params_full}")

    # Final training on full data
    model_full = build_mtlr_model(
        input_dim=X_data.shape[1],
        time_bins=time_bins_data,
        params=best_params_full,
    )
    optimizer_full = get_optimizer(model_full, best_params_full)

    model_full, losses_full = train_mtlr_model(
        model=model_full,
        optimizer=optimizer_full,
        X_tensor=X_tensor,
        y_time_fold=y_time_data,
        y_event_fold=y_event_data,
        time_bins=time_bins_data,
        C1=best_params_full["C1"],
        n_epochs=best_params_full["n_epochs"],
        clip_grad=1.0,
        verbose=True,
    )

    return model_full, time_bins_data, best_params_full, losses_full

In [ ]:
# Train 5 final models on the full train set
X_tensor_full = torch.tensor(X.values, dtype=torch.float32)
X_eval_tensor = torch.tensor(X_eval.values, dtype=torch.float32)

models_final = []
risk_matrix_eval = []

print("\n=== Final ensemble training (5 models) on the full train set ===")

for m in range(N_MODELS_ENSEMBLE):
    base_seed = 5000 + m
    print(f"\n##### Final model {m+1}/{N_MODELS_ENSEMBLE} (seed={base_seed}) #####")
    model_m, time_bins_full, best_params_m, losses_m = optimize_and_train_full_model(
        X_data=X, y_time_data=y_time, y_event_data=y_event, base_seed=base_seed
    )

    if model_m is None:
        print("⚠ Final model is invalid, ignored.")
        continue

    models_final.append(model_m)
    risk_eval_m = mtlr_risk_scores(model_m, X_eval_tensor)
    risk_matrix_eval.append(risk_eval_m)

In [ ]:
risk_matrix_eval = np.array(risk_matrix_eval)
print(f"\nNumber of valid final models in ensemble: {risk_matrix_eval.shape[0]}")

if risk_matrix_eval.shape[0] == 0:
    raise RuntimeError("No valid final model for the ensemble.")

# Final ensemble: rank-based aggregation
risk_ensemble_eval = rank_based_ensemble(risk_matrix_eval)

# Optional normalization to [0, 1] for interpretability
risk_min = risk_ensemble_eval.min()
risk_max = risk_ensemble_eval.max()
risk_ensemble_eval_norm = (risk_ensemble_eval - risk_min) / (risk_max - risk_min + 1e-8)

print(f"Min risk (raw ensemble) : {risk_min:.4f}")
print(f"Max risk (raw ensemble) : {risk_max:.4f}")

In [ ]:
# Build prediction dataframe
submission = pd.DataFrame({
    "ID": df_eval["ID"],
    "risk_score": risk_ensemble_eval_norm,  # or risk_ensemble_eval if you prefer raw scores
})

print(submission.head())

submission.to_csv("submission_mtlr_ensemble_nestedcv.csv", index=False)
print("\nPrediction file saved as submission_mtlr_ensemble_nestedcv.csv")